In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("../new_data.csv")
df.English = df.English.str.replace('\n', ' ')
df=df.dropna(axis=0)
df.head()

,Id,Enunciado,Tópico,Contexto,English
0,994,Meu primeiro programa\nEscreva um programa que...,Ambientação,Outros,"""My first show Write a program that prints the..."
1,996,Impressão de caracteres na tela\nEscreva um pr...,Ambientação,Outros,Screen character printing Write a program tha...
2,999,Impressão de caracteres na tela (Bart Simpson)...,Ambientação,Outros,Screen Character Printing (Bart Simpson) Writ...
3,1000,Operadores aritméticos\nQual o valor de X para...,Ambientação,Matemático,Arithmetic Operators What is the value of X s...
4,578,Operadores aritméticos \nQual o valor de Y par...,Ambientação,Matemático,Arithmetic Operators What is the value of Y s...


In [3]:
contador=0
lista=[]
while(contador<len(df['English'])):
    if(df['Contexto'][contador]=='Outros'):
        lista+=[contador]
    contador+=1
df=df.drop(lista)

In [4]:
import spacy
nlp = spacy.load('en')

In [5]:
master=[]
for elemento in df['English']:
    doc=nlp(elemento)
    lista=[]
    for token in doc:
        if(token.text not in nlp.Defaults.stop_words and token.lemma_ not in nlp.Defaults.stop_words and token.text.isalpha()):
            lista+=[token.lemma_]
    master+=[' '.join(lista)]

In [6]:
df['tokens']=master

In [7]:
X = df['tokens'].values
y = df['Contexto'].values

In [8]:
class GloveVectorizer:
    def __init__(self):
        # load in pre-trained word vectors
        print('Loading word vectors...')
        word2vec = {}
        embedding = []
        idx2word = []
        with open('../../../../cursos/pln/novo_pln_deep-learning/nlp_class2/myCodes/glove/glove.6B.300d.txt') as f:
            # is just a space-separated text file in the format:
            # word vec[0] vec[1] vec[2] ...
            for line in f:
                values = line.split()
                word = values[0]
                vec = np.asarray(values[1:], dtype='float32')
                word2vec[word] = vec
                embedding.append(vec)
                idx2word.append(word)
        print('Found %s word vectors.' % len(word2vec))

        # save for later
        self.word2vec = word2vec
        self.embedding = np.array(embedding)
        self.word2idx = {v:k for k,v in enumerate(idx2word)}
        self.V, self.D = self.embedding.shape

    def fit(self, data):
        pass

    def transform(self, data):
        X = np.zeros((len(data), self.D))
        n = 0
        emptycount = 0
        for sentence in data:
            tokens = sentence.lower().split()
            vecs = []
            for word in tokens:
                if word in self.word2vec:
                    vec = self.word2vec[word]
                    vecs.append(vec)
            if len(vecs) > 0:
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis=0)
            else:
                emptycount += 1
            n += 1
        print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
        return X

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [8]:
from gensim.models import KeyedVectors
class Word2VecVectorizer:
    def __init__(self):
        print("Loading in word vectors...")
        self.word_vectors = KeyedVectors.load_word2vec_format('../../../../cursos/pln/novo_pln_deep-learning/nlp_class2/myCodes/google/GoogleNews-vectors-negative300.bin',binary=True)
        print("Finished loading in word vectors")

    def fit(self, data):
        pass

    def transform(self, data):
        # determine the dimensionality of vectors
        v = self.word_vectors.get_vector('king')
        self.D = v.shape[0]

        X = np.zeros((len(data), self.D))
        n = 0
        emptycount = 0
        for sentence in data:
            tokens = sentence.split()
            vecs = []
            m = 0
            for word in tokens:
                try:
                    # throws KeyError if word not found
                    vec = self.word_vectors.get_vector(word)
                    vecs.append(vec)
                    m += 1
                except KeyError:
                    pass
            if len(vecs) > 0:
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis=0)
            else:
                emptycount += 1
            n += 1
        print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
        return X


    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [9]:
##Professor, aqui é a escolha do tokenizer, o restante segue igual para os dois

# vectorizer = GloveVectorizer()
vectorizer = Word2VecVectorizer()

Loading in word vectors...
Finished loading in word vectors


In [109]:
from sklearn.model_selection import StratifiedKFold
vetor_X_train=[]
vetor_y_train=[]
vetor_X_test=[]
vetor_y_test=[]
split = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
for train_index, val_index in split.split(X, y):
    vetor_X_train+=[X[train_index]]
    vetor_X_test+=[X[val_index]]
    vetor_y_train+=[y[train_index]]
    vetor_y_test+=[y[val_index]]

/home/samuel/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [110]:
contador=0
predictions=[]
while(contador<len(vetor_X_train)):
    X_train=vetor_X_train[contador]
    y_train=vetor_y_train[contador]
    X_test=vetor_X_test[contador]
    y_test=vetor_y_test[contador]
    
    X_train = vectorizer.fit_transform(X_train)
    estimator = SVC(kernel='rbf')
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(vectorizer.fit_transform(X_test))
    predictions+=[y_pred]

    contador+=1

Numer of samples with no words found: 0 / 887
Numer of samples with no words found: 0 / 99
Numer of samples with no words found: 0 / 887
Numer of samples with no words found: 0 / 99
Numer of samples with no words found: 0 / 887
Numer of samples with no words found: 0 / 99
Numer of samples with no words found: 0 / 887
Numer of samples with no words found: 0 / 99
Numer of samples with no words found: 0 / 887
Numer of samples with no words found: 0 / 99
Numer of samples with no words found: 0 / 887
Numer of samples with no words found: 0 / 99
Numer of samples with no words found: 0 / 888
Numer of samples with no words found: 0 / 98
Numer of samples with no words found: 0 / 888
Numer of samples with no words found: 0 / 98
Numer of samples with no words found: 0 / 888
Numer of samples with no words found: 0 / 98
Numer of samples with no words found: 0 / 888
Numer of samples with no words found: 0 / 98


In [111]:
y_pred=[]
for lista1 in predictions:
    for elemento1 in lista1:
        y_pred+=[elemento1]

In [112]:
y_test_sla=[]
for lista2 in vetor_y_test:
    for elemento2 in lista2:
        y_test_sla+=[elemento2]

In [113]:
from sklearn import metrics

In [114]:
print(metrics.confusion_matrix(y_test_sla,y_pred))

[[ 32   2   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0
    0   0   0   0   0]
 [  3 113   0   0   0   1   0   0   0   0   0   1   1   0   0   0   0   0
    0   0   0   0   1]
 [  0   2   0   0   0   0   0   0   0   0   0   0   4   0   0   0   0   0
    0   0   0   0   0]
 [  0   5   0  11   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   4   0   0  15   0   0   0   0   0   0   1   1   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   2  73   0   0   0   1   0   0   3   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   2   0  40   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0  29   0   0   0   0   1   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  31   0   0   1   3   0   0   0   0   0
    0   0   0   0   1]
 [  0   0   0   0   0   0   0   1   0   8   0   1   1   0   0   0   0   0
    0   0   0   0   0]
 [  0   1   0   0   0   0   0 

In [115]:
print(metrics.classification_report(y_test_sla,y_pred))

                 precision    recall  f1-score   support

       Bancário       0.78      0.91      0.84        35
      Comercial       0.76      0.94      0.84       120
  Computacional       0.00      0.00      0.00         6
        Consumo       1.00      0.69      0.81        16
    Data e hora       0.68      0.71      0.70        21
        Escolar       0.97      0.92      0.95        79
        Esporte       1.00      0.95      0.98        42
Filmes e séries       0.97      0.97      0.97        30
         Física       0.94      0.86      0.90        36
      Geografia       0.89      0.73      0.80        11
        Imposto       1.00      0.80      0.89         5
          Jogos       0.91      0.88      0.89        96
     Matemático       0.91      1.00      0.95       261
  Meio ambiente       1.00      0.89      0.94        18
       Pesquisa       1.00      0.78      0.88        18
         Pessoa       1.00      0.91      0.95        22
      População       0.88    

/home/samuel/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [116]:
print(metrics.accuracy_score(y_test_sla,y_pred))

0.9016227180527383


In [117]:
np.savetxt('ytrue_W2V+SVC_Context-Others', y_test_sla, fmt='%s')
np.savetxt('ypred_W2V+SVC_Context-Others', y_pred, fmt='%s')

In [22]:
def noOthers(df):
    contador=0
    lista=[]
    while(contador<len(df['English'])):
        if(df['Contexto'][contador]=='Outros'):
            lista+=[contador]
        contador+=1
    return df.drop(lista)

In [23]:
##Aqui eu pego os dados com parafrase, esses dados ja estão com o pre-processamento
dfs_train = []
dfs_test = []
for i in range(0, 10):
    df_train = pd.read_csv('../dadosParafrase/treino'+str(i+1)+'.csv')
    df_train=noOthers(df_train)
    dfs_train.append(df_train)
    df_test = pd.read_csv('../dadosParafrase/teste'+str(i+1)+'.csv')
    df_test=noOthers(df_test)
    dfs_test.append(df_test)

In [118]:
contador=0
predictions=[]
vetor_y_test=[]
while(contador<len(dfs_train)):
    X_train=dfs_train[contador]['master'].values
    y_train=dfs_train[contador]['Contexto'].values
    X_test=dfs_test[contador]['master'].values
    vetor_y_test+=[dfs_test[contador]['Contexto'].values]
        
    X_train = vectorizer.fit_transform(X_train)
    estimator = SVC(kernel='rbf')
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(vectorizer.fit_transform(X_test))
    predictions+=[y_pred]

    contador+=1

Numer of samples with no words found: 0 / 2870
Numer of samples with no words found: 0 / 99
Numer of samples with no words found: 0 / 2875
Numer of samples with no words found: 0 / 99
Numer of samples with no words found: 0 / 2875
Numer of samples with no words found: 0 / 99
Numer of samples with no words found: 0 / 2875
Numer of samples with no words found: 0 / 99
Numer of samples with no words found: 0 / 2875
Numer of samples with no words found: 0 / 99
Numer of samples with no words found: 0 / 2880
Numer of samples with no words found: 0 / 98
Numer of samples with no words found: 0 / 2880
Numer of samples with no words found: 0 / 98
Numer of samples with no words found: 0 / 2875
Numer of samples with no words found: 0 / 98
Numer of samples with no words found: 0 / 2875
Numer of samples with no words found: 0 / 98
Numer of samples with no words found: 0 / 2866
Numer of samples with no words found: 0 / 99


In [119]:
y_pred=[]
for lista1 in predictions:
    for elemento1 in lista1:
        y_pred+=[elemento1]

In [120]:
y_test_sla=[]
for lista2 in vetor_y_test:
    for elemento2 in lista2:
        y_test_sla+=[elemento2]

In [121]:
print(metrics.confusion_matrix(y_test_sla,y_pred))

[[ 32   2   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0
    0   0   0   0   0]
 [  3 112   0   0   0   1   0   0   0   0   0   1   1   0   0   0   0   0
    0   0   0   0   2]
 [  0   0   6   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   1   0  15   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   1   0   0  18   0   0   0   0   0   0   0   1   0   0   0   0   0
    0   0   0   0   1]
 [  0   0   0   0   1  73   1   0   0   1   0   0   3   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0  42   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0  30   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  34   0   0   1   0   0   0   0   0   0
    0   0   0   0   1]
 [  0   0   0   0   0   0   0   1   0   9   0   0   1   0   0   0   0   0
    0   0   0   0   0]
 [  0   1   0   0   0   0   0 

In [122]:
print(metrics.classification_report(y_test_sla,y_pred))

                 precision    recall  f1-score   support

       Bancário       0.82      0.91      0.86        35
      Comercial       0.86      0.93      0.90       120
  Computacional       1.00      1.00      1.00         6
        Consumo       1.00      0.94      0.97        16
    Data e hora       0.86      0.86      0.86        21
        Escolar       0.97      0.92      0.95        79
        Esporte       0.95      1.00      0.98        42
Filmes e séries       0.94      1.00      0.97        30
         Física       0.94      0.94      0.94        36
      Geografia       0.82      0.82      0.82        11
        Imposto       1.00      0.80      0.89         5
          Jogos       0.97      0.89      0.92        96
     Matemático       0.96      0.99      0.98       261
  Meio ambiente       1.00      0.89      0.94        18
       Pesquisa       1.00      0.89      0.94        18
         Pessoa       1.00      0.95      0.98        22
      População       0.96    

In [123]:
print(metrics.accuracy_score(y_test_sla,y_pred))

0.9432048681541582


In [91]:
np.savetxt('ytrue_W2V+GaussianNB_Context-Others', y_test_sla, fmt='%s')
np.savetxt('ypred_W2V+GaussianNB_Context-Others', y_pred, fmt='%s')

NameError: name 'y_test_sla' is not defined